In [1]:
# import libraries
import torch
from pathlib import Path

from neuralhydrology.nh_run import start_run, eval_run
import pickle
from neuralhydrology.evaluation.plots import percentile_plot_plotly

### Train model with negative logarithm of the likelihood

In [ ]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path("1_basin.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("1_basin.yml"), gpu=-1)

### Evaluate run on test set
The path to the best model is automatically saved in the variable best_model, and the corresponding best epoch is stored as a string in best_epoch.

In [2]:
run_dir = Path("runs/single_basin_multi_forcing_mdn_2208_162034")
eval_run(run_dir=run_dir, period="test")

# Evaluation: 100%|██████████| 1/1 [00:03<00:00,  3.40s/it]


### Load and inspect model predictions
Load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [3]:
with open(run_dir / "test" / "model_epoch050" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

dict_keys(['DE4'])

In [4]:
results['DE4']['1D']['xr']['discharge_vol_sim']

<xarray.DataArray 'discharge_vol_sim' (date: 823, time_step: 1, samples: 1000)>
array([[[0.12291986, 0.10457563, 0.11484581, ..., 0.12379169,
         0.12491441, 0.10685939]],

       [[0.16596198, 0.16069728, 0.11485946, ..., 0.13561177,
         0.11779475, 0.13658893]],

       [[0.15427563, 0.1402505 , 0.1431874 , ..., 0.14753759,
         0.14386383, 0.14481646]],

       ...,

       [[0.35980916, 0.31895185, 0.33022735, ..., 0.2756355 ,
         0.32338175, 0.3444524 ]],

       [[0.34016   , 0.3475133 , 0.2094068 , ..., 0.29305923,
         0.2814126 , 0.3480337 ]],

       [[0.30686232, 0.27849773, 0.307238  , ..., 0.22178888,
         0.33446264, 0.28720927]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2018-10-01 2018-10-02 ... 2020-12-31
  * time_step  (time_step) int64 0
Dimensions without coordinates: samples

In [5]:
# Extract observations and simulations
qobs = results['DE4']['1D']['xr']['discharge_vol_obs']
qsim = results['DE4']['1D']['xr']['discharge_vol_sim']

In [7]:
fig = percentile_plot_plotly(qobs, qsim, title="Percentile Plot - Test Period")
fig.show()